# Лабораторная работа 1.5
Трунов Антон М8О-304Б Вариант 16

# Задание
Реализовать алгоритм QR – разложения матриц в виде программы. На его основе разработать программу, реализующую QR – алгоритм решения полной проблемы собственных значений произвольных матриц, задавая в качестве входных данных матрицу и точность вычислений. С использованием разработанного программного обеспечения найти собственные значения матрицы.
\begin{pmatrix}
1 & 2 & 5 \\
-8 & 0 & 6 \\
7 & -9 & -7 \\
\end{pmatrix}

In [1]:
import numpy as np

from math import sqrt

e = 1
def machine_epsilon():
    global e
    while e + 1 != 1:  # calculating machine epsilon
        e = e / 2

def householder(x):
    N = x.shape[0]
    y = np.zeros(N)
    xNorm = sqrt(np.inner(x,x))#computing 2-norm of the vector x
    y[0] = xNorm
    u = np.copy(x)  #constructing vector u
    if x[0] > 0 and xNorm - x[0] < e: #to escape cancellations
        u[0] = -np.inner(x[1:],x[1:])/(x[0]+xNorm)
    else:
        u[0] = x[0] - xNorm
    u[:] = u[:] / sqrt(np.inner(u, u))
    return u

def qT_matrix(U):
    Q = np.eye(U.shape[1]) #Q.T = (1-u_1*u_1T)*...*(1-u_n*u_nT)*1
    for i in range(U.shape[0]-1,-1,-1):#can be formed via iterations as (1-u_i*(u_iT*QT_i+1))
        for j in range (U.shape[1]):
            Q[:, j] = Q[:, j] - 2 * np.inner(U[i,:], Q[:, j]) * U[i,:]
    return Q


def qr_decomp(a):
    r = np.array(a, copy=True, dtype=float)
    m, n = r.shape
    N = n if m > n else m-1
    U = np.zeros((n, m))
    for i in range(N):
        u = np.zeros(m)
        u[i:] = householder(r[i:,i]) #householder func which returns vector u
        U[i,:] = u[:] #matrix of u vectors
        for j in range(n):
            r[:,j] = r[:,j] - 2*np.inner(u,r[:,j])*u[:]#A-2u(u.T*A)
    q = qT_matrix(U)
    return q ,r

def qr_algorithm(A,eps=1e-5,maxiter=500):
    niter = 0
    norm = 1
    while norm > eps and niter < maxiter:
        LamPrev = A[0,0]
        Q, R = qr_decomp(A)
        A = R @ Q
        norm = abs(A[0, 0] - LamPrev)
        niter += 1
    return A, niter

def eigenval(A,eps=1e-4):
    arr = np.empty(A.shape[0],dtype=complex)
    i = 0
    while i < A.shape[0]:
        if abs(A[i+1,i]) < eps:
            arr[i] = A[i,i]
        else:
            coef = [1,-A[i,i]-A[i+1,i+1],A[i,i]*A[i+1,i+1]-A[i,i+1]*A[i+1,i]]
            rot=np.roots(coef)
            arr[i] = rot[0]
            arr[i+1] = rot[1]
            i+=1
        i+=1
    return arr


In [2]:
machine_epsilon()
a = np.array([[1,3,1,6,7],[1,1,4,5,9],[4,3,1,2,9],[4,7,-7,0,11],[7,9,3,1,8]],dtype=float)
print(a)
A, niter = qr_algorithm(np.copy(a))
eigenA = eigenval(A)
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
print("Eigenvalues:",eigenA,"\nNumber of iterations:",niter)
print("Via linalg:",np.linalg.eig(a)[0])


[[ 1.  3.  1.  6.  7.]
 [ 1.  1.  4.  5.  9.]
 [ 4.  3.  1.  2.  9.]
 [ 4.  7. -7.  0. 11.]
 [ 7.  9.  3.  1.  8.]]
Eigenvalues: [22.035+0.j    -6.176+3.486j -6.176-3.486j  0.659+1.925j  0.659-1.925j] 
Number of iterations: 13
Via linalg: [22.035+0.j    -6.176+3.486j -6.176-3.486j  0.659+1.925j  0.659-1.925j]


In [3]:
n = 6
a = np.array([[1 if abs(i-j)==1 else 0 for i in range(n)] for j in range(n)]).reshape(n,n)+2*np.eye(n)
c = np.concatenate((np.zeros((n,n)),np.eye(n)),axis=1)
b = np.concatenate((-100*a,-a),axis=1)
A = np.concatenate((c,b),axis=0)
A, niter = qr_algorithm(np.copy(A))
eigenA = eigenval(A)
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
print("Eigenvalues:",np.array(eigenA).reshape(12,1),"\nNumber of iterations:",niter)
print("Via linalg:",np.linalg.eig(a)[0])

Eigenvalues: [[-1.901+19.406j]
 [-1.901-19.406j]
 [-1.624+17.946j]
 [-1.624-17.946j]
 [-1.223+15.589j]
 [-1.223-15.589j]
 [-0.777+12.446j]
 [-0.777-12.446j]
 [-0.377 +8.67j ]
 [-0.377 -8.67j ]
 [-0.099 +4.449j]
 [-0.099 -4.449j]] 
Number of iterations: 500
Via linalg: [3.802 3.247 2.445 0.198 1.555 0.753]


[[-1.901+19.406j]


 [-1.901-19.406j]
 
 
 [-1.623+17.946j]
 
 
 [-1.623-17.946j]
 
 
 [-1.223+15.589j]
 
 
 [-1.223-15.589j]
 
 
 [-0.777+12.446j]
 
 
 [-0.777-12.446j]
 
 
 [-0.377 +8.67j ]
 
 
 [-0.377 -8.67j ]
 
 
 [-0.099 +4.449j]
 
 
 [-0.099 -4.449j]]